In [7]:
!pip install -U langgraph langchain  openai huggingface_hub --quiet
!pip install -U langchain-openai --quiet
!pip install -U langchain-community --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [8]:
import openai
import os
from google.colab import userdata


openai.api_key =userdata.get('OPEN_API_KEY')

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_metrics():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    return df

def detect_anomalies(df):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1]


In [10]:
from typing import List, TypedDict, Set
from langgraph.graph import StateGraph, END
import hashlib

class AIOpsState(TypedDict):
    chat_history: List[dict]
    anomalies: str
    seen_anomalies: Set[str]


In [11]:
def aiops_langchain_agent(state: AIOpsState) -> AIOpsState:
    from langchain.chains import LLMChain
    from langchain.prompts import PromptTemplate

    history = state["chat_history"]
    prompt = PromptTemplate.from_template(
        """Detected anomalies in system metrics:\n\n{anomalies}
Explain possible causes, system impact, and recommended actions."""
    )
    chain = LLMChain(llm=llm, prompt=prompt)

    result = chain.run(anomalies=state["anomalies"])
    history.append({"role": "user", "content": prompt.format(anomalies=state["anomalies"])})
    history.append({"role": "assistant", "content": result})

    return {
        "chat_history": history,
        "anomalies": state["anomalies"],
        "seen_anomalies": state["seen_anomalies"]
    }


In [12]:
import hashlib

def deduplicate_event(state: AIOpsState) -> AIOpsState:
    history = state["chat_history"]
    hash_id = hashlib.md5(state["anomalies"].encode()).hexdigest()

    if hash_id in state["seen_anomalies"]:
        history.append({"role": "user", "content": "Duplicate anomaly detected. Skipping."})
        history.append({"role": "assistant", "content": "Anomaly already processed earlier."})
    else:
        state["seen_anomalies"].add(hash_id)

    return {
        "chat_history": history,
        "anomalies": state["anomalies"],
        "seen_anomalies": state["seen_anomalies"]
    }


In [13]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", temperature=0.3, openai_api_key=openai.api_key)
def aiops_agent(state: AIOpsState) -> AIOpsState:
    history = state.get("chat_history", [])

    # Append new prompt with current anomaly
    prompt = {
        "role": "user",
        "content": f"""Detected anomalies in system metrics:\n\n{state['anomalies']}\n
Explain the possible causes, impacts, and fixes."""
    }
    history.append(prompt)

    # Call ChatGPT
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=history
    )
    reply = response.choices[0].message.content.strip()

    # Save model response
    history.append({"role": "assistant", "content": reply})

    return {"chat_history": history, "anomalies": state["anomalies"]}


/tmp/ipython-input-13-1066192067.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0.3, openai_api_key=openai.api_key)


In [14]:
from langgraph.graph import StateGraph, END

builder = StateGraph(state_schema=AIOpsState)
builder.add_node("deduplicate_event", deduplicate_event)
builder.add_node("aiops_langchain_agent", aiops_langchain_agent)

builder.set_entry_point("deduplicate_event")
builder.add_edge("deduplicate_event", "aiops_langchain_agent")
builder.add_edge("aiops_langchain_agent", END)

graph = builder.compile()




In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_anomalies():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1].to_string(index=False)


In [16]:
# Persistent memory state
state = {
    "chat_history": [],
    "seen_anomalies": set()
}

# Simulate 2 anomaly events (same data to test deduplication)
for i in range(2):
    print(f"\n Run {i+1}")
    anomaly_str = generate_anomalies()
    input_state = {
        "chat_history": state["chat_history"],
        "anomalies": anomaly_str,
        "seen_anomalies": state["seen_anomalies"]
    }
    state = graph.invoke(input_state)

    for msg in state["chat_history"][-2:]:
        print(f"[{msg['role'].upper()}]: {msg['content']}\n")



 Run 1


/tmp/ipython-input-11-3375322124.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-11-3375322124.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(anomalies=state["anomalies"])


[USER]: Detected anomalies in system metrics:

       cpu     memory    disk_io  anomaly
222.084508 217.853534 262.411635       -1
231.252957 245.023815 269.386036       -1
225.339937 227.214063 291.348540       -1
226.897169 239.585032 285.333456       -1
223.484506 248.074496 269.922889       -1
Explain possible causes, system impact, and recommended actions.

[ASSISTANT]: Possible Causes:
1. High CPU usage: This could be due to a large number of processes running simultaneously, a single process consuming a large amount of CPU, or a CPU-intensive task such as data processing or video encoding.
2. High Memory usage: This could be due to memory leaks in applications, too many applications running simultaneously, or large data sets being processed.
3. High Disk IO: This could be due to large amounts of data being read from or written to the disk, such as during a backup operation, large file transfers, or heavy database operations.

System Impact:
1. High CPU usage can cause the system